<a href="https://colab.research.google.com/github/axdget/test1/blob/master/vk_interview_2023_10_19_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание для кандидата



---



---



*! Необходимо сперва запустить 2 ячейки снизу*


In [ ]:
# @title
!pip install numpy pandas randomtimestamp random-user-agent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.4 MB/s eta 0:00:00


In [ ]:
# @title
import random
import re

import numpy as np
import pandas as pd
import sqlite3
from randomtimestamp import randomtimestamp
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

np.random.seed(42)
pd.set_option('display.width', 1000)


def generate_dates(start, end, n, seed=42, replace=True):
    dates = pd.date_range(start, end).to_series()
    return dates.sample(n, replace=replace, random_state=seed)

def insert_df_into_table(df, table_name, db='mydb.sqlite'):
    conn = sqlite3.connect(db)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

def get_select_query(query, db = 'mydb.sqlite'):
    connection = sqlite3.connect(db)
    res = pd.read_sql_query(query, connection)
    return res


# task №3
df_1 = \
    pd.DataFrame({\
                  'user_id':\
                  np.random.choice(int(2e2), int(1e3), replace=True),\
                  'date':\
                  generate_dates('20230101', '20230630', int(1e3)).values\
                  })\
                  .sort_values(by='date')

insert_df_into_table(df_1, 'dau')


# task №2
n_rows = 100

df = \
    pd.DataFrame({\
                  'id':\
                  [idx + 1 for idx in range(n_rows)],\
                  'employee_id':\
                  np.random.choice(int(2e2), n_rows, replace=True),\
                  'salary':\
                  np.int32(np.abs(np.random.normal(loc=5e2, scale=2e2, size=n_rows))) * int(1e2),\
                  'department_id':\
                  np.random.randint(0, 5, n_rows) + 1
                  })\
                  .sort_values(by='id')

insert_df_into_table(df, 'employee')

df = \
    pd.DataFrame({\
                  'id':\
                  [idx + 1 for idx in range(5)],\
                  'name':\
                  ['Backend', 'Fronted', 'QA', 'Data Science', 'Devops'],
                  })\
                  .sort_values(by='id')

insert_df_into_table(df, 'department')


# task №1
n_rows = 100

df = \
    pd.DataFrame({\
                  'number':\
                  np.int32(np.abs(np.random.normal(loc=5e2, scale=5e1, size=150))),
                  })

insert_df_into_table(df, 'numbers')


# task №4
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value, OperatingSystem.MACOS.value]
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=20)
user_agents = user_agent_rotator.get_user_agents()

pat = '\([^\)]*(Windows|Linux|Mac OS)+[^\)]*\)'
list_usag = [usag for usag in filter(lambda x: bool(re.search(pat, x)), [user_agent_rotator.get_random_user_agent() for usag in range(100)])]
list_usag = [' '.join(i.split(' ')[-2:]) + ' ' + ' '.join(i.split(' ')[:-2])  for i in list_usag]

user_agents_df = pd.DataFrame({'user_agent': list_usag, 'count_launch': np.random.randint(1e5, 1e6, len(list_usag))})

user_agents_df.user_agent = [col.replace('(', str(random.randint(1, 100)) + ' (') for col in user_agents_df.user_agent]
user_agents_df.user_agent = [col.replace(') AppleWebKit', ') ' + str(random.randint(1, 100)) + ' AppleWebKit') for col in user_agents_df.user_agent]



---



---



## Блок 1. Общие вопросы

1. Что такое ошибка 1-го и 2-го рода? В чем идея статистического вывода?
2. Какой смысл следующих метрик: MAU / Retention rate / Timespent / ARPU?
3. Какие статистические критерии ты знаешь и чем они отличаются, какие у них области применения?
3. Есть 2 таблицы c 10 и 20 строк соотвественно. Сколько строк будет минимально / максимально при INNER JOIN / LEFT JOIN / OUTER JOIN / CROSS JOIN


## Блок 2. SQL

**1. Есть таблица `numbers`, в которой содержатся некоторые числовые значения. Необходимо достать максимальное значение, которое содержится только 1 раз в таблице.**

In [ ]:
query = """

select number,c
from(select
  number,count (number) as c
from
  numbers
group by number
having count (number) =1) f
order by number desc
limit 1
"""

get_select_query(query)

,number,c
0,631,1


**2. Есть таблица `employee`, в которой содержатся инфо о размере дохода каждого сотрудника, в таблице `department` содержатся словарь наименований отделов. Хотим узнать у кого самый большой доход в каждом департаменте.**

In [ ]:
query = """

with max_sel as(select *
from (select
   department_id,employee_id,salary, max(salary) over (partition by department_id) as max_sal
from
  employee) f
where salary = max_sal)

select *
from max_sel ms
left join department d on ms.department_id = d.id
"""

get_select_query(query)

,department_id,employee_id,salary,max_sal,id,name
0,1,89,85600,85600,1,Backend
1,2,100,81600,81600,2,Fronted
2,3,20,76000,76000,3,QA
3,4,177,100700,100700,4,Data Science
4,5,139,102800,102800,5,Devops


In [ ]:
query = """

select
  *
from
  department
limit
  5

"""

get_select_query(query)

,id,name
0,1,Backend
1,2,Fronted
2,3,QA
3,4,Data Science
4,5,Devops


**3. Допустим у нас есть логи по заходам пользователей в приложение в таблице dau:**

|user_id|date|
|---|---|
|2123|2023-06-01|
|312312|2023-06-02|
|38129|2023-06-02|
|213102|2023-06-02|
|...|...|
|214123|2023-09-30|

Необходимо написать запрос, который получает количество удержанных пользователей за каждый отчетный месяц. Данный параметр определяется как количество пользователей, которые зашли повторно в приложение в следующем месяце.
На выходе хотим получим таблицу следующего вида (значения ниже условные):


---


|month_dt|uniq_users_current_month|uniq_users_next_month|reten_rate|
|---|---|---|---|
|2023-01-01|100|90|0.9|
|2023-02-01|200|150|0.75|
|...|...|...|...|

In [ ]:
query = """

with all_id as (select
 user_id,DATE(date, 'start of month') as month
from
  dau)

select month,count(distinct user_id) as uniq_users_current_month
from all_id
group by month
"""

get_select_query(query)

,month,count(distinct user_id)
0,2023-01-01,114
1,2023-02-01,109
2,2023-03-01,108
3,2023-04-01,117
4,2023-05-01,123
5,2023-06-01,114


## Блок 3. Python

**1. Есть 2 списка `a` и `b`. Нужно написать функцию, которая посчитает сколько уникальных элементов `a` входят в `b`**

Пример:

a = [1, 2, 3, 3]

b = [3, 4, 5]

Функция должна вернуть значение `1`

In [ ]:
def uniq(a,b):
    c= 0
    for i in set(a):
      if i in set(b):
        c+=1
    return c
a = [1, 2, 3, 3]

b = [3, 4, 5]
result = uniq(a,b)
print(result)

1


**2. Есть датафрейм `user_agents_df`, в котором содержатся логи по кол-ву заходов в приложение по каждому юзер агенту. Необходимо посчитать суммарное кол-во заходов по каждой платформе.**

Платформы находятся внутри круглых скобок, где есть инфо об ОС (Windows, Linux, Mac OS), нужно вытащить все, что внутри этих скобок:

*Chrome/40.0.2214.111 Safari/537.36,gzip20 (gfe) Mozilla/5.0 20 (`Windows NT 6.0`) 48 AppleWebKit/537.36 20 (KHTML, like Gecko)*

*Chrome/36 Safari/538 Mozilla/5.0 58 (`Windows NT 6.1; WOW64`) 31 AppleWebKit/538 58 (KHTML, like Gecko)*

In [ ]:
import regex as re

In [ ]:
s= 'Chrome/40.0.2214.111 Safari/537.36,gzip20 (gfe) Mozilla/5.0 20 (Windows NT 6.0) 48 AppleWebKit/537.36 20 (KHTML, like Gecko)'
print (s[s.find('Windows')-3:])

0 (Windows NT 6.0) 48 AppleWebKit/537.36 20 (KHTML, like Gecko)


In [ ]:
user_agents_df.head(5)
def reg(x):
    n = None
    l =['Windows', 'Linux', 'Mac OS']
    for i in l:
      s = re.search(r' \(.*\) ',x[x.find(i)-3:])
      if type(s) is not type(n):
        break

    return s
#(\([a-zA-Z]\))
user_agents_df['os_info'] = user_agents_df['user_agent'].apply(reg)

In [ ]:
user_agents_df

,user_agent,count_launch,os_info
0,Chrome/35.0.1916.150 Safari/537.36 Mozilla/5.0...,427304,"<regex.Match object; span=(1, 26), match=' (Wi..."
1,Chrome/42.0.0.9757 Safari/537.36 Mozilla/5.0 7...,904034,"<regex.Match object; span=(1, 26), match=' (Wi..."
2,Chrome/40.0.2214.114 Safari/537.36 Mozilla/5.0...,465464,"<regex.Match object; span=(1, 26), match=' (Wi..."
3,Chrome/50.0.2638.0 Safari/537.36 Mozilla/5.0 2...,713089,"<regex.Match object; span=(1, 31), match=' (Wi..."
4,Chrome/50.0.2638.0 Safari/537.36 Mozilla/5.0 2...,388075,"<regex.Match object; span=(1, 31), match=' (Wi..."
...,...,...,...
91,Chrome/50.0.2638.0 Safari/537.36 Mozilla/5.0 6...,263038,"<regex.Match object; span=(1, 31), match=' (Wi..."
92,Chrome/32.0.2019.89 Safari/536.24 Mozilla/5.0 ...,268383,"<regex.Match object; span=(1, 26), match=' (Wi..."
93,Chrome/46.0.2490.86 Safari/537.14 Mozilla/5.0 ...,918007,"<regex.Match object; span=(1, 19), match=' (Wi..."
94,Safari/537.36 2345Explorer/9.4.3.17934 Mozilla...,425996,"<regex.Match object; span=(1, 70), match=' (Wi..."
